In [1]:
# Bibliotecas
import pandas as pd
import spacy
import nltk


nlp = spacy.load("pt_core_news_sm")

# Dados (IMDB traduzido)

In [2]:
imdb = pd.read_csv("../dados/imdb_traduzido.csv", sep="|")

In [3]:
print("Número de sentimentos positivos:",
      len(imdb[imdb['sentimento']==1]))
print("Número de sentimentos negativos:",
      len(imdb[imdb['sentimento']==0]))

Número de sentimentos positivos: 500
Número de sentimentos negativos: 500


In [4]:
imdb

,texto,sentimento
0,"Um muito, muito, muito lento, filme sem rumo s...",0
1,Não tenho certeza que foi mais perdido - os pe...,0
2,A tentativa artiness com ângulos de câmera pre...,0
3,Muito pouco de música ou qualquer coisa para f...,0
4,A melhor cena do filme foi quando Gerardo está...,1
5,"O resto do filme carece de arte, encanto, o qu...",0
6,Desperdiçado duas horas.,0
7,Viu o filme hoje e pensei que era um esforço b...,1
8,Um pouco previsível.,0
9,Adorei a fundição de Jimmy Buffet como o profe...,1


# Normalização de texto

In [5]:
# Transformar para minúsculas
# Remoção de stopwords
# Rotulação de partes da fala
#     - Manter: substantivos, nomes próprios, adjetivos, verbos, advérbios
#     - Alternativa 1: melhora só com substantivos, nomes próprios, adjetivos e advérbios?
#     - Alternativa 2: melhora só com adjetivos e advérbios?
# Redução das palavras (estemização ou lematização é melhor?)
# Remoção de palavras de baixa frequência (< 3)
# Remoção de palavras que aparecem na maioria dos documentos(> 50%)
# Extra: identificar entidades nomeadas (imprimir na tela)

In [6]:
imdb['texto'] = imdb['texto'].map(str.lower)

In [7]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def remover_stopwords(t):
    return " ".join([p for p in t.split() if p not in stopwords])
    
imdb['texto'] = imdb['texto'].map(remover_stopwords)

In [8]:
def manter_pos(t):
    doc = nlp(t)
    palavras_ok = [p.text for p in doc\
                   if p.pos_ in\
#                        ['ADJ', 'ADV']]
                        ['NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV']]
    return " ".join(palavras_ok)

imdb['texto'] = imdb['texto'].map(manter_pos)

# Treinamento e teste + Representação

In [9]:
# Textos e classes
X = list(imdb['texto'])
y = list(imdb['sentimento'])

# Vocabulário
vocabulario = set()
for x in X:
    for p in x.split():
        vocabulario.add(p)
vocabulario = list(vocabulario)
vocabulario.sort()

In [10]:
from sklearn.model_selection import train_test_split

textos_trein, textos_teste, y_trein, y_teste = train_test_split(X, y, test_size=0.3)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vetorizador_tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2), vocabulary=vocabulario)
X_trein = vetorizador_tfidf.fit_transform(textos_trein)
X_teste = vetorizador_tfidf.transform(textos_teste)

# Classificador

## Treinamento

In [12]:
from sklearn.neural_network import MLPClassifier

classificador = MLPClassifier(alpha=0.1, max_iter=1000)
classificador.fit(X_trein, y_trein)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

## Avaliação

In [13]:
score = classificador.score(X_teste, y_teste)

In [14]:
score

0.7533333333333333

# Exemplo

In [43]:
texto_exemplo = "Esse filme é terrível, com atores ruins e cenas mal filmadas"
#texto_exemplo = "Filmagens muito boas, e a atriz principal é espetacular"
texto_exemplo = texto_exemplo.lower()
texto_exemplo = remover_stopwords(texto_exemplo)
texto_exemplo = manter_pos(texto_exemplo)
print(texto_exemplo)
x = vetorizador_tfidf.transform([texto_exemplo])
print("Predição:", classificador.predict(x))
print("Probabilidades:", classificador.predict_proba(x))

filme terrível atores ruins cenas mal filmadas
Predição: [0]
Probabilidades: [[0.9945656 0.0054344]]
